# StackOverFlow 

In [80]:
# general imports
# please make sure you have gensim (if you don't use: pip install gensim)
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import sys
import nltk
nltk.download('stopwords')
# sys.path.append("C:/Users/Gal/PycharmProjects/GroundInstances/")
# from create_user_profile import *
# from clean_data import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# classification imports 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [64]:
# Project definition
project_dir = '/Users/Gal/Documents/Repositories/Workshop-in-Data-Science/'

## Load data

In [65]:
# Generating the final dataframe with all the fetures might take several hours,
# Here we use the basic dataframes we got from Kaggle to show you how we extracted the relevant features
from clean_data import clean_data

clean_data(project_dir + '/data/Answers.csv', project_dir + '/data/Clean_Answers.csv')
clean_data(project_dir + '/data/Questions.csv', project_dir + '/data/Clean_Questions.csv')
clean_data(project_dir + '/data/Tags.csv', project_dir + '/data/Clean_Tags.csv')

ans = pd.read_csv(project_dir + '/data/Clean_Answers.csv')
qus = pd.read_csv( project_dir + '/data/Clean_Questions.csv')
tags = pd.read_csv( project_dir + '/data/Clean_Tags.csv')

##  First Observation on the data

In [5]:
ans.head(5)

,Id,OwnerUserId,CreationDate,ParentId,Score,IsAcceptedAnswer,Body
0,79741,3259.0,2008-09-17T03:43:22Z,79709,-1,False,<p>It's tough to say definitively without know...
1,79768,6043.0,2008-09-17T03:48:29Z,79709,9,False,<p>use variables in the outer function instead...
2,79779,8002.0,2008-09-17T03:49:36Z,79709,0,False,<p>Third approach: inner function returns a re...
3,79827,14257.0,2008-09-17T03:58:26Z,79709,1,False,"<p>I'm not sure I understand the question, but..."
4,79893,14928.0,2008-09-17T04:11:08Z,79709,6,False,"<p>Remember your Knuth. ""Premature optimizati..."


In [6]:
qus.head(5)

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,77434,14008.0,2008-09-16T21:40:29Z,171,How to access the last value in a vector?,<p>Suppose I have a vector that is nested in a...
1,95007,15842.0,2008-09-18T17:59:19Z,56,Explain the quantile() function in R,<p>I've been mystified by the R quantile funct...
2,255697,1941213.0,2008-11-01T15:48:30Z,4,Is there an R package for learning a Dirichlet...,<p>I'm looking for a an <code>R</code> package...
3,359438,2173.0,2008-12-11T14:02:06Z,4,Optimization packages for R,<p>Does anyone know of any optimization packag...
4,439526,37751.0,2009-01-13T15:58:48Z,23,Thinking in Vectors with R,<p>I know that R works most efficiently with v...


In [7]:
tags.head(5)

,Id,Tag
0,77434,vector
1,79709,memory
2,79709,function
3,79709,global-variables
4,79709,side-effects


In [75]:
import main
main.main()

cleaning data and save it back to the same directory
reading it back...
Split to train and test and prepare user data_frame from train and add columns for the NLP features
saving data frames
applying gensim models on train and test data frames


IndexError: index 223050 is out of bounds for axis 1 with size 223050

In [66]:
# DO NOT EXCUTE THIS - it's add a lot of feature - it's better using the prepared Datafram next cell
from clean_data import split_to_train_test
train_df, test_df = split_to_train_test(qus, ans, tags)

nlp_features.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[new_col] = df[orig_col].apply(clean_column)


saving data frames


In [17]:
train_df = pd.read_csv(project_dir + 'data/clean_data/train_with_tag_clusters_and_user_scores.csv')
test_df = pd.read_csv(project_dir + 'data/clean_data/test_with_tag_clusters_and_user_scores.csv')

## Creating users list and graph

In [18]:
# This is also might take a time ~ 2 hours
from create_user_profile import *
users = list_of_users(train_df)
users.create_user_graph()
users.set_node_properties()
users.set_user_scores(train_df)

In [19]:
#saving the objects
user_df = users.generate_data_frame(project_dir + "\user_dataframe.csv")
users.save_obj(project_dir + "\user_list.pkl")

In [ ]:
#loading objects
user_df = pd.read_csv('C:\Users\Gal\Documents\Library-Science\user_dataframe.csv')
user_df.head()

In [ ]:
user_df[user_df['neg_questions'] > 0]

## User's graph analysis

In [21]:
G = list_of_users.load_obj(project_dir  + 'user_list.pkl')

#### devide to sub graphs and some details of the graph

In [22]:
print len(list(nx.weakly_connected_component_subgraphs(G.graph))), G.graph.number_of_edges(), G.graph.number_of_nodes()
sub_graphs = list(nx.weakly_connected_component_subgraphs(G.graph))

2425 157144 53956


#### Largest components - most of the users are connected in the same component

In [23]:
sorted([len(x) for x in sub_graphs])[-10:]

[5, 5, 5, 5, 5, 5, 6, 6, 8, 50018]

## classification

In [59]:
# classification imports 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [74]:
# NN tests are in different module since we need tensorflow enviornment 
train_df = pd.read_csv(project_dir + '/data/train_df.csv')
test_df = pd.read_csv(project_dir + '/data/test_df.csv')

train_df.fillna(0, inplace=True)
test_df.fillna(0, inplace=True)

# define all features
basic_features = ['Score_ans', 'Score_qus']
nlp_features = ['topic_' + str(i) + "_qus" for i in range(10)] + ['topic_' + str(i) + "_ans" for i in range(10)]
time_features = ['diff_percentile_bucket', 'hirerchy']
tags_features = ["cluster_" + str(i) for i in range(50)] + ["user_score_cluster_" + str(i) for i in range(50)]\
                + ["user_score_general","total_score_user_question_by_clusters", "total_score_user_question_by_clusters_relative"]
users_features = ['neg_answers_user', 'neg_questions_user', 'number_of_answers_user', 'out_degree_user', 'question_total_score_user',
                 'answer_total_score_user', 'closeness_centrality_user', 'in_degree_user']

all_features = basic_features + users_features + time_features # + nlp_features # + tags_features
target = 'IsAcceptedAnswer'

# training - random forest
random_forest_clf = RandomForestClassifier(n_estimators=300, max_depth=5, random_state=0)
random_forest_clf.fit(train_df[all_features], train_df[target])
results_random_forest = random_forest_clf.predict(test_df[all_features])

correct_predictions = np.sum(results_random_forest == np.array(test_df[target]))

print "results of random forest:"
print (correct_predictions) / float(len(results_random_forest))

# training - logistic regression
# Iterated over parameters: pentalty {l1, l2}
logisticRegr = LogisticRegression(max_iter=5000, class_weight='balanced', penalty='l2')
logisticRegr.fit(train_df[all_features], train_df[target])
correct_predictions = np.sum(logisticRegr.predict(test_df[all_features]) == test_df[target])

print "results of logistic regression:"
print correct_predictions / float(len(test_df[target]))


# training - SVM 
# clf = svm.SVC()
# clf.fit(train_df[all_features], train_df[target]) 
# correct_predictions = np.sum(clf.predict(test_df[all_features]) ==  test_df[target])

# print "results of SVM:"
# print correct_predictions / float(len(test_df[target]))


df loaded


KeyError: "['diff_percentile_bucket' 'hirerchy'] not in index"

In [42]:
np.sum(test_df['IsAcceptedAnswer'] == results_random_forest)

49181

In [ ]:
np.count_nonzero(ans["IsAcceptedAnswer"]) / float(len(ans))

In [ ]:
import collections
c = collections.Counter(tags["Id"])
c

# Ans score - IsAccepted graph

In [ ]:
%matplotlib inline
F = [x for x in ans[ans['IsAcceptedAnswer'] == False]['Score'] if x > -10 or x<50]
T = [x for x in ans[ans['IsAcceptedAnswer'] == True]['Score'] if x > -10 or x<50]
bins = np.linspace(-10, 20, 30)

plt.hist(F, bins, alpha=0.5, label='Not accepted')
plt.hist(T, bins, alpha=0.5, label='Accepted', weights=[len(T)/float(len(F))]*len(T))

plt.legend(loc='upper right')
plt.xlabel('score')
plt.ylabel('# samples')

plt.show()

In [ ]:
len(ans), len(qus)

In [ ]:
T.head()

In [ ]:
ans[ans['IsAcceptedAnswer'] == False]['ParentId'].nunique(), len(ans[ans['IsAcceptedAnswer'] == False]['ParentId'])

## NLP FEATURES

#### cleaning data

In [3]:
import nltk
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
print stopWords

set([u'all', u'just', u'being', u'over', u'both', u'through', u'yourselves', u'its', u'before', u'o', u'hadn', u'herself', u'll', u'had', u'should', u'to', u'only', u'won', u'under', u'ours', u'has', u'do', u'them', u'his', u'very', u'they', u'not', u'during', u'now', u'him', u'nor', u'd', u'did', u'didn', u'this', u'she', u'each', u'further', u'where', u'few', u'because', u'doing', u'some', u'hasn', u'are', u'our', u'ourselves', u'out', u'what', u'for', u'while', u're', u'does', u'above', u'between', u'mustn', u't', u'be', u'we', u'who', u'were', u'here', u'shouldn', u'hers', u'by', u'on', u'about', u'couldn', u'of', u'against', u's', u'isn', u'or', u'own', u'into', u'yourself', u'down', u'mightn', u'wasn', u'your', u'from', u'her', u'their', u'aren', u'there', u'been', u'whom', u'too', u'wouldn', u'themselves', u'weren', u'was', u'until', u'more', u'himself', u'that', u'but', u'don', u'with', u'than', u'those', u'he', u'me', u'myself', u'ma', u'these', u'up', u'will', u'below', u'ain

In [1]:
import gensim
Lda = gensim.models.ldamodel.LdaModel
from gensim import corpora

C:\Users\Gal\Anaconda2\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
# this how to generate topics for questions and answers - also it might take a while... 
from nlp_features import *
train_df = pd.read_csv(project_dir + '/data/clean_data/train_with_tag_clusters_and_user_scores.csv')
q_docs = train_df['clean_qus']
a_docs = train_df['clean_ans']

q_docs = [x.split(" ") for x in q_docs if type(x)!=float]
a_docs = [x.split(" ") for x in a_docs if type(x)!=float]

q_dictionary = corpora.Dictionary(q_docs)
q_doc_term_matrix = [q_dictionary.doc2bow(doc) for doc in q_docs]

a_dictionary = corpora.Dictionary(a_docs)
a_doc_term_matrix = [a_dictionary.doc2bow(doc) for doc in a_docs]

Lda = gensim.models.ldamodel.LdaModel

print "building a model for questions"
q_ldamodel = Lda(q_doc_term_matrix, num_topics=10, id2word=q_dictionary, passes=50)
q_ldamodel.save("my_lda_model_qus_10_topics")

print "building a model for answers"
a_ldamodel = Lda(a_doc_term_matrix, num_topics=10, id2word=a_dictionary, passes=50)
a_ldamodel.save("my_lda_model_ans_10_topics")

building a model for questions
building a model for answers


#### veryfining that topics make sense

In [12]:
for x in a_ldamodel.print_topics(num_topics=10, num_words=20):
    print x

(0, u'0.017*"~" + 0.014*"number" + 0.014*"matrix" + 0.014*"data" + 0.011*"time" + 0.011*"values" + 0.010*"vector" + 0.010*"model" + 0.010*"value" + 0.008*"element" + 0.008*"length" + 0.006*"first" + 0.005*"test" + 0.005*"calculate" + 0.005*"two" + 0.005*"random" + 0.005*"mean" + 0.005*"returns" + 0.005*"function" + 0.005*"/"')
(1, u'0.103*"1" + 0.098*"2" + 0.072*"3" + 0.062*"na" + 0.053*"4" + 0.041*"5" + 0.033*"b" + 0.031*"6" + 0.021*"7" + 0.021*"8" + 0.020*"c" + 0.018*"10" + 0.016*"9" + 0.010*"x" + 0.010*"$" + 0.010*"12" + 0.009*"11" + 0.007*"id" + 0.007*"13" + 0.006*"15"')
(2, u'0.042*"image" + 0.041*"description" + 0.035*"*" + 0.023*"plot" + 0.021*"x" + 0.014*"color" + 0.012*"size" + 0.012*"fill" + 0.011*"labels" + 0.010*"add" + 0.010*"max" + 0.010*"/" + 0.010*"mean" + 0.010*"col" + 0.009*"label" + 0.008*"min" + 0.008*"p" + 0.008*"width" + 0.007*"points" + 0.007*"use"')
(3, u'0.019*"function" + 0.019*"use" + 0.012*"like" + 0.012*"would" + 0.011*"want" + 0.010*"need" + 0.010*"code" +

#### verifying that scores to each doc makes sense

In [14]:
for i in range(10):
    bow = a_dictionary.doc2bow(a_docs[i])
    t = a_ldamodel.get_document_topics(bow)
    print t

[(3, 0.70084924), (8, 0.28314927)]
[(0, 0.39112002), (1, 0.025), (2, 0.02500137), (3, 0.40887052), (4, 0.025), (5, 0.025), (6, 0.025002552), (7, 0.025), (8, 0.025005588), (9, 0.025)]
[(0, 0.12699741), (2, 0.22358847), (3, 0.49486497), (7, 0.10000015)]
[(0, 0.21933755), (3, 0.38200584), (8, 0.3448088)]
[(1, 0.40886503), (7, 0.4306464), (8, 0.12714858)]
[(0, 0.17579961), (1, 0.49311706), (2, 0.045494415), (3, 0.25570112), (5, 0.020795628)]
[(0, 0.18219188), (1, 0.014285768), (2, 0.014286166), (3, 0.7035199), (4, 0.014285714), (5, 0.014285714), (6, 0.014286555), (7, 0.014285775), (8, 0.014286356), (9, 0.014286191)]
[(0, 0.07689339), (3, 0.54721534), (4, 0.061111346), (8, 0.28143942)]
[(3, 0.6876724), (6, 0.2395936)]
[(3, 0.8194172), (8, 0.1570521)]


### Some properties of NLP features

In [90]:
train_df = pd.read_csv(project_dir + 'data/clean_data/train_with_tag_clusters_and_user_scores.csv')
accept_df = train_df [train_df ['IsAcceptedAnswer'] == 1]
reject_df = train_df [train_df ['IsAcceptedAnswer'] == 0]

D_accept = find_correlations_in_topics(accept_df)
D_reject = find_correlations_in_topics(reject_df)

KeyError: u"None of [['topic_0_qus_Ans', 'topic_1_qus_Ans', 'topic_2_qus_Ans', 'topic_3_qus_Ans', 'topic_4_qus_Ans', 'topic_5_qus_Ans', 'topic_6_qus_Ans', 'topic_7_qus_Ans', 'topic_8_qus_Ans', 'topic_9_qus_Ans']] are in the [index]"

In [87]:
ans_topics = ['topic_0_ans', 'topic_1_ans', 'topic_2_ans', 'topic_3_ans','topic_4_ans', 'topic_5_ans',
                  'topic_6_ans', 'topic_7_ans', 'topic_8_ans', 'topic_9_ans']
qus_topics = ['topic_0_qus', 'topic_1_qus','topic_2_qus', 'topic_3_qus', 'topic_4_qus', 'topic_5_qus',
                  'topic_6_qus', 'topic_7_qus', 'topic_8_qus', 'topic_9_qus']

for i, row in train_df.iterrows():
    ans_index = np.argmax(list(row[ans_topics]))
    qus_index = np.argmax(list(row[qus_topics]))
    print ans_index, qus_index


0
2 5
1
0 5
2
2 5
3
0 5
4
2 5
5
2 5
6
2 5
7
2 5
8
2 5
9
2 5
10


## Some basic feature analysis

In [ ]:
# In this section we will analyse some of the feature statistics after creating them all
train_df = pd.read_csv(project_dir + '/data/clean_data/train_with_tag_clusters_and_user_scores.csv')